In [12]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

## 2.1

In [13]:
list_of_files = ['proj3_data1.json', 'proj3_data2.json', 'proj3_data3.json']
df = pd.DataFrame()

# load the files
for file in list_of_files:
    df = df.append(pd.read_json(file))

# reset indexes
df.reset_index(inplace=True, drop=True)
print(df.to_string())

# save to json
df.to_json('ex01_all_data.json')

          make     model  body_type  doors  top_speed  acceleration  fuel_consumption                  engine
0         Audi        A3  hatchback    5.0        220           7.2               6.0                1.5 TFSI
1          BMW  3 Series      sedan    4.0        240           6.2               NaN    2.0L TwinPower Turbo
2   Volkswagen      Golf  hatchback    5.0        210           8.0               5.0                 1.5 TSI
3      Renault      Clio  hatchback    5.0        190          11.0               6.5                 1.5 dCi
4         Fiat       500  hatchback    3.0        160          12.9               NaN             0.9 TwinAir
5      Peugeot       208  hatchback    NaN        190           9.9               5.0            1.2 PureTech
6        Volvo      XC60        SUV    5.0        220           8.0               6.0                      B4
7         Seat      Leon  hatchback    5.0        240           8.0               NaN                 1.5 TSI
8         

/tmp/ipykernel_70955/1414121679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_json(file))
/tmp/ipykernel_70955/1414121679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_json(file))
/tmp/ipykernel_70955/1414121679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_json(file))


## 2.2

In [14]:
import csv
list_of_columns_with_missing_values = []
for column in df.columns:
    # Calculate number of missing values
    number_of_missing_values = df[column].isnull().sum()
    if number_of_missing_values > 0:
        # append if there are any
        list_of_columns_with_missing_values.append((column, number_of_missing_values))


with open('ex01_no_nulls.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file)
    # Data rows
    for row in list_of_columns_with_missing_values:
        writer.writerow(row)

## 2.3

In [15]:
import json
with open('proj3_params.json') as json_file:
    data = json.load(json_file)
print(data)

# Add new column
df['description'] = df[data['concat_columns']].apply(lambda x: ' '.join(x), axis=1)
print(df.to_string())

# save to json
df.to_json('ex03_descriptions.json')

{'concat_columns': ['make', 'model', 'engine'], 'id_vars': ['make', 'model'], 'join_column': 'engine', 'int_columns': ['doors', 'displacement', 'horsepower', 'cylinders'], 'grouping_column': 'make', 'pivot_index': 'make', 'pivot_columns': 'fuel_type', 'pivot_values': 'fuel_consumption', 'aggregations': [['displacement', 'min'], ['displacement', 'max'], ['fuel_consumption', 'mean']]}
          make     model  body_type  doors  top_speed  acceleration  fuel_consumption                  engine                        description
0         Audi        A3  hatchback    5.0        220           7.2               6.0                1.5 TFSI                   Audi A3 1.5 TFSI
1          BMW  3 Series      sedan    4.0        240           6.2               NaN    2.0L TwinPower Turbo  BMW 3 Series 2.0L TwinPower Turbo
2   Volkswagen      Golf  hatchback    5.0        210           8.0               5.0                 1.5 TSI            Volkswagen Golf 1.5 TSI
3      Renault      Clio  hatchbac

## 2.4

In [16]:
# load json to dataframe
df_json = pd.read_json('proj3_more_data.json')
    
list_of_columns_to_join_on = data['join_column']
print(list_of_columns_to_join_on)

# join the data
df = df.merge(df_json, on=list_of_columns_to_join_on, how='left')
print(df.to_string())
df.to_json('ex04_joined.json')

engine
          make     model  body_type  doors  top_speed  acceleration  fuel_consumption                  engine                        description  displacement  horsepower fuel_type  cylinders emissions_class
0         Audi        A3  hatchback    5.0        220           7.2               6.0                1.5 TFSI                   Audi A3 1.5 TFSI        1498.0       150.0  gasoline        4.0         Euro 6d
1          BMW  3 Series      sedan    4.0        240           6.2               NaN    2.0L TwinPower Turbo  BMW 3 Series 2.0L TwinPower Turbo        1998.0       184.0  gasoline        4.0         Euro 6d
2   Volkswagen      Golf  hatchback    5.0        210           8.0               5.0                 1.5 TSI            Volkswagen Golf 1.5 TSI        1498.0       150.0  gasoline        4.0         Euro 6d
3      Renault      Clio  hatchback    5.0        190          11.0               6.5                 1.5 dCi               Renault Clio 1.5 dCi        1461.0   

## 2.5

In [17]:
# Drop description column, save to json
for index, row in df.iterrows():
    description = row['description'].lower().replace(' ', '_')
    row = row.drop(['description']).to_dict()

    for key, value in row.items():
        if pd.isna(value):
            row[key] = None
    print("1 ", row)
    # save to json
    filename = f"ex05_{description}.json"
    with open(filename, 'w') as outfile:
        json.dump(row, outfile)
    
    # ----------------------------
    # int columns
    int_columns = data['int_columns']
    for col in int_columns:
        if pd.notnull(row[col]):
            row[col] = int(row[col])
        else:
            row[col] = None
        
    # change NaN to null
    for key, value in row.items():
        if pd.isna(value):
            row[key] = None
    
    print("2 ", row)
    # save to json
    filename = f"ex05_int_{description}.json"
    with open(filename, 'w') as outfile:
        json.dump(row, outfile)
    

1  {'make': 'Audi', 'model': 'A3', 'body_type': 'hatchback', 'doors': 5.0, 'top_speed': 220, 'acceleration': 7.2, 'fuel_consumption': 6.0, 'engine': '1.5 TFSI', 'displacement': 1498.0, 'horsepower': 150.0, 'fuel_type': 'gasoline', 'cylinders': 4.0, 'emissions_class': 'Euro 6d'}
2  {'make': 'Audi', 'model': 'A3', 'body_type': 'hatchback', 'doors': 5, 'top_speed': 220, 'acceleration': 7.2, 'fuel_consumption': 6.0, 'engine': '1.5 TFSI', 'displacement': 1498, 'horsepower': 150, 'fuel_type': 'gasoline', 'cylinders': 4, 'emissions_class': 'Euro 6d'}
1  {'make': 'BMW', 'model': '3 Series', 'body_type': 'sedan', 'doors': 4.0, 'top_speed': 240, 'acceleration': 6.2, 'fuel_consumption': None, 'engine': '2.0L TwinPower Turbo', 'displacement': 1998.0, 'horsepower': 184.0, 'fuel_type': 'gasoline', 'cylinders': 4.0, 'emissions_class': 'Euro 6d'}
2  {'make': 'BMW', 'model': '3 Series', 'body_type': 'sedan', 'doors': 4, 'top_speed': 240, 'acceleration': 6.2, 'fuel_consumption': None, 'engine': '2.0L Tw

## 2.6

In [18]:
aggregations_list = data['aggregations']
print(aggregations_list)

aggregations_dict = {}

for col, agg_func in aggregations_list:
    key = f"{agg_func}_{col}"
    # apply aggregation function to the columns
    aggregations_dict[key] = df[col].agg(agg_func)
print(aggregations_dict)

with open('ex06_aggregations.json', 'w') as f:
    json.dump(aggregations_dict, f)

[['displacement', 'min'], ['displacement', 'max'], ['fuel_consumption', 'mean']]
{'min_displacement': 875.0, 'max_displacement': 3000.0, 'mean_fuel_consumption': 6.414285714285714}


## 2.7

In [19]:
# Group
grouping_column = data['grouping_column']
grouped_df = df.groupby(grouping_column).mean()

# Mean values
mean_df = grouped_df.select_dtypes(include=np.number).agg('mean')

# get groups conainig more than one record 
mean_df = mean_df[mean_df > 1]
mean_df = mean_df.to_frame() 

# save
mean_df.to_csv('ex07_groups.csv')
mean_df


/tmp/ipykernel_70955/2300238153.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_df = df.groupby(grouping_column).mean()


,0
doors,4.366667
top_speed,226.363636
acceleration,7.936364
fuel_consumption,6.850000
displacement,1739.227273
horsepower,214.000000
cylinders,3.954545


## 2.8

In [20]:
reshape1_df = df.groupby([data["pivot_index"], data["pivot_columns"]])[data["pivot_values"]].max()
# pivot to desired shape
reshape1_df = reshape1_df.reset_index().pivot(index=data["pivot_index"], columns=data["pivot_columns"], values=data["pivot_values"])
# remove rows with all nulls
reshape1_df = reshape1_df[reshape1_df.notnull().any(axis=1)]
print(reshape1_df.to_string())

# save to json
reshape1_df.to_pickle('ex08_pivot.pkl')

fuel_type   diesel  gasoline  hybrid
make                                
Audi           NaN       6.0     NaN
Ford           NaN      10.2     NaN
Opel           NaN       5.5     NaN
Peugeot        NaN       5.6     NaN
Porsche        NaN      11.8     NaN
Renault        6.5       NaN     NaN
Volkswagen     6.0       5.0     NaN
Volvo          NaN       NaN     6.6


In [21]:
# Transform wide df to long one
reshape2_df = pd.melt(df, id_vars=data["id_vars"], var_name='variable', value_name='value')

# sort
reshape2_df = reshape2_df.sort_values(by=data["id_vars"])

# remove the index
reshape2_df = reshape2_df.reset_index(drop=True)

reshape2_df.to_csv('ex08_melt.csv')
print(reshape2_df.to_string())



           make     model          variable                              value
0          Audi        A3         body_type                          hatchback
1          Audi        A3             doors                                5.0
2          Audi        A3         top_speed                                220
3          Audi        A3      acceleration                                7.2
4          Audi        A3  fuel_consumption                                6.0
5          Audi        A3            engine                           1.5 TFSI
6          Audi        A3       description                   Audi A3 1.5 TFSI
7          Audi        A3      displacement                             1498.0
8          Audi        A3        horsepower                              150.0
9          Audi        A3         fuel_type                           gasoline
10         Audi        A3         cylinders                                4.0
11         Audi        A3   emissions_class         

In [22]:
reshape3_df = pd.read_csv('proj3_statistics.csv')
print(reshape3_df.to_string())

pivot_index = data['pivot_index']

# use wide_to_long
reshape3_df = pd.wide_to_long(reshape3_df, reshape3_df.columns[1:].str.extract(r'^([^_]+)_', expand = False).unique(), i = reshape3_df.columns[0], j = 'Prefix', sep = '_')

# save to pkl
reshape3_df.to_pickle('ex08_stats.pkl')

   Country  Audi_2019  Audi_2020  BMW_2019  BMW_2020  Volkswagen_2019  Volkswagen_2020  Renault_2019  Renault_2020
0   Poland         12         14        21        25               32               36            22            20
1  Germany         24         26        31        35               44               48            17            15
2   France         20         22        29        33               36               40            28            26
3    Spain         16         18        25        29               40               44            19            17
4    Italy         18         20        27        31               48               52            31            29
